## E-COMMERCE RECOMMENDER SYSTEM

In [1]:
# Loading the imports
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
# Loading the data
df = pd.read_csv('./FlikartProductSales.csv')
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,image_links
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...,https://rukminim1.flixcart.com/image/612/612/x...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...,https://rukminim1.flixcart.com/image/612/612/x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/x...
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/k...
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...,https://rukminim1.flixcart.com/image/612/612/j...
...,...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,₹380,₹999,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,NaN,https://rukminim1.flixcart.com/image/612/612/x...
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,₹179,₹299,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...,https://rukminim1.flixcart.com/image/612/612/k...
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,₹354,₹600,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...,https://rukminim1.flixcart.com/image/612/612/k...
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,₹498,₹599,Foodie Puppies Group,4.5,NaN,For Fish Flavor: NA Food Type: Dry Suitable Fo...,https://rukminim1.flixcart.com/image/612/612/j...


In [3]:
df.shape

(12041, 12)

In [4]:
df.dtypes

category_1         object
category_2         object
category_3         object
title              object
product_rating    float64
selling_price      object
mrp                object
seller_name        object
seller_rating     float64
description        object
highlights         object
image_links        object
dtype: object

### Data Cleaning

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   category_1      12041 non-null  object 
 1   category_2      12041 non-null  object 
 2   category_3      12041 non-null  object 
 3   title           12023 non-null  object 
 4   product_rating  11961 non-null  float64
 5   selling_price   12013 non-null  object 
 6   mrp             11666 non-null  object 
 7   seller_name     11825 non-null  object 
 8   seller_rating   11827 non-null  float64
 9   description     5021 non-null   object 
 10  highlights      6560 non-null   object 
 11  image_links     12041 non-null  object 
dtypes: float64(2), object(10)
memory usage: 1.1+ MB


In [6]:
df.describe()

,product_rating,seller_rating
count,11961.000000,11827.000000
mean,4.061617,4.394538
std,0.345575,0.404288
min,1.000000,3.000000
25%,3.900000,4.100000
50%,4.100000,4.400000
75%,4.300000,4.700000
max,5.000000,5.000000


In [7]:
df.columns

Index(['category_1', 'category_2', 'category_3', 'title', 'product_rating',
       'selling_price', 'mrp', 'seller_name', 'seller_rating', 'description',
       'highlights', 'image_links'],
      dtype='object')

In [8]:
# Dropping the unnecessary columns
df.drop(['image_links'],axis=1,inplace=True)
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...
...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,₹380,₹999,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,NaN
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,₹179,₹299,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,₹354,₹600,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,₹498,₹599,Foodie Puppies Group,4.5,NaN,For Fish Flavor: NA Food Type: Dry Suitable Fo...


In [9]:
# Checking null values
df.isnull().sum()

category_1           0
category_2           0
category_3           0
title               18
product_rating      80
selling_price       28
mrp                375
seller_name        216
seller_rating      214
description       7020
highlights        5481
dtype: int64

In [10]:
# Inscpecting the title column
df[df['title'].isnull()]

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
310,"Sports, Books and More",Exercise Fitness,Cardio Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681,"Sports, Books and More",Food Essentials,Chocolates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1578,"Sports, Books and More",Industrial &Scientific tools,Packaging and Shipping Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,Electronics,Mobile Accessories,Screenguards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2164,Electronics,Smart Wearable Tech,Smart Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3557,Men's wear,Bottom Wear,Casual Trousers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4516,Men's wear,Innerwear and loungewear,Night Suits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4664,Men's wear,Watches,Titan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4776,Men's wear,Accessories,Backpacks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4867,Men's wear,Accessories,Belts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Filling the null value with "other"
df['title'].fillna('Other',inplace=True)
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...
...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,₹380,₹999,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,NaN
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,₹179,₹299,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,₹354,₹600,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,₹498,₹599,Foodie Puppies Group,4.5,NaN,For Fish Flavor: NA Food Type: Dry Suitable Fo...


In [12]:
df.title.isnull().sum()

0

In [13]:
# Checking the Product Rating column
df[df['product_rating'].isnull()]

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
310,"Sports, Books and More",Exercise Fitness,Cardio Equipment,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,"Sports, Books and More",Food Essentials,Nuts and Dry Fruits,HERBALIFE Weight Loss Combo pack Formula 1 + P...,NaN,"₹2,710","₹4,476",NaN,NaN,Formula 1 Nutri nutritional requirements. Pack...,vitamins minerals protein
681,"Sports, Books and More",Food Essentials,Chocolates,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,"Sports, Books and More",Food Essentials,Gifting Combos,HERBALIFE Weight Loss Combo pack Formula 1 + P...,NaN,"₹2,710","₹4,476",NaN,NaN,Formula 1 Nutri nutritional requirements. Pack...,vitamins minerals protein
1438,"Sports, Books and More",Industrial &Scientific tools,Industrial Measurement Devices,DTTOOLS 25~50mm Micrometer Outside Screw Gauge...,NaN,₹889,"₹1,099",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11420,Home and Furniture,Home Improvement,Lawn and Gardening,Femily Organic STONE DUST powder for Plants Ma...,NaN,₹189,₹199,NaN,NaN,Calcium carbonate is used to help prevent the ...,"Manure, Potting Mixture, Fertilizer Used for A..."
11548,Home and Furniture,Home Décor,Clocks,BAREPEPE Digital Black Clock,NaN,NaN,NaN,NaN,NaN,NaN,Alarm Height x Width 100 mm x 160 mm Digital
11792,Home and Furniture,Home Lighting,Table Lamp,"META White Table Lamp Table Lamp (56 cm, White)",NaN,"₹2,999","₹5,000",NaN,NaN,"White Table lamp for home,bedroom,living room,...","Color: White, Gold Power Source: AC Adapter W ..."
11817,Home and Furniture,Home Lighting,Ceiling Lamp,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Replacing Nulls with 0 
df['product_rating'].fillna('0',inplace=True)
df['product_rating'].isnull().sum()


0

In [15]:
# Inspecting the selling price column
df[df['selling_price'].isnull()]

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
310,"Sports, Books and More",Exercise Fitness,Cardio Equipment,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
681,"Sports, Books and More",Food Essentials,Chocolates,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
1578,"Sports, Books and More",Industrial &Scientific tools,Packaging and Shipping Products,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
1936,Electronics,Mobile Accessories,Screenguards,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
2156,Electronics,Smart Wearable Tech,Smart Watches,"alt OG Bluetooth Calling, 1.69"" HD Display wit...",4.0,NaN,NaN,NaN,NaN,With the amazing features of the alt OG smartw...,1.69� TFT LCD screen (240*280 px) Bluetooth ca...
2164,Electronics,Smart Wearable Tech,Smart Watches,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
2653,Electronics,Speakers,DTH Set Top Box,"Dish TV HD DTH, Hindi Family Saver Pack 1 Mont...",3.0,NaN,NaN,NaN,NaN,This offer includes DishTV HD set top box Conn...,NaN
3123,Men's wear,Foot Wear,Sneakers,BRUTON Lightweight Pack Of 1 Trendy Sneakers S...,3.7,NaN,NaN,NaN,NaN,NaN,NaN
3557,Men's wear,Bottom Wear,Casual Trousers,Other,0,NaN,NaN,NaN,NaN,NaN,NaN
4516,Men's wear,Innerwear and loungewear,Night Suits,Other,0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df['selling_price'].fillna('0',inplace=True)
df['selling_price'].isnull().sum()

0

In [17]:
# M
df[df['mrp'].isnull()]
df['mrp'].fillna('0',inplace=True)
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...
...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,₹380,₹999,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,NaN
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,₹179,₹299,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,₹354,₹600,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,₹498,₹599,Foodie Puppies Group,4.5,NaN,For Fish Flavor: NA Food Type: Dry Suitable Fo...


In [18]:
columns_to_clean = ['selling_price', 'mrp']

# Remove the rupee sign and commas, then convert to numeric for each column
for column in columns_to_clean:
    df[column] = df[column].replace({'₹': '', ',': ''}, regex=True).astype(float)

# Example conversion rate from INR to KES
conversion_rate = 0.76  

# Convert the Selling Price and Original Price from INR to KES
df['selling_price'] = df['selling_price'] * conversion_rate
df['mrp'] = df['mrp'] * conversion_rate
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,1227.40,3040.00,I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,115.52,456.00,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,44.84,151.24,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,57.00,227.24,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,250.04,1063.24,IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...
...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,288.80,759.24,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,NaN
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,136.04,227.24,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,269.04,456.00,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,378.48,455.24,Foodie Puppies Group,4.5,NaN,For Fish Flavor: NA Food Type: Dry Suitable Fo...


In [19]:
# Rename columns
df.rename(columns={
    'selling_price': 'selling_price(KSH.)',
    'mrp': 'mrp(KSH.)'
}, inplace=True)

In [20]:
# Inspecting the seller_name 
df[df['seller_name'].isnull()]

,category_1,category_2,category_3,title,product_rating,selling_price(KSH.),mrp(KSH.),seller_name,seller_rating,description,highlights
40,"Sports, Books and More",Sports,Badminton,VICTOO SPORTS VICTOO PROFESIONAL BADMINTON Bad...,3.8,254.60,456.00,NaN,NaN,NaN,"Sport Type: Badminton Ideal For: Boys, Girls, ..."
310,"Sports, Books and More",Exercise Fitness,Cardio Equipment,Other,0,0.00,0.00,NaN,NaN,NaN,NaN
600,"Sports, Books and More",Food Essentials,Nuts and Dry Fruits,OPEN SECRET Snacks Combo | Nutty Cookies and N...,2.7,345.80,0.00,NaN,NaN,Nutty Spread - Choco Almond TASTY & HEALTHY: T...,NaN
607,"Sports, Books and More",Food Essentials,Nuts and Dry Fruits,HERBALIFE Weight Loss Combo pack Formula 1 + P...,0,2059.60,3401.76,NaN,NaN,Formula 1 Nutri nutritional requirements. Pack...,vitamins minerals protein
681,"Sports, Books and More",Food Essentials,Chocolates,Other,0,0.00,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11792,Home and Furniture,Home Lighting,Table Lamp,"META White Table Lamp Table Lamp (56 cm, White)",0,2279.24,3800.00,NaN,NaN,"White Table lamp for home,bedroom,living room,...","Color: White, Gold Power Source: AC Adapter W ..."
11817,Home and Furniture,Home Lighting,Ceiling Lamp,Other,0,0.00,0.00,NaN,NaN,NaN,NaN
11882,Home and Furniture,Festive Decor and Gifts,Festive Decor and Gifts,SHANKAR PERFUMEERY WORKS Combo Pack - Aromatic...,4.1,209.00,0.00,NaN,NaN,NaN,Type: Incense Sticks Fragrance: A fresh fragra...
11904,Home and Furniture,Festive Decor and Gifts,Festive Decor and Gifts,UNIBIC Season's Greetings|Christmas & New Year...,3.6,455.24,759.24,NaN,NaN,"Season's Greetings, Christmas & New Year Gift,...","Includes: Cake, Chocolate, Cookies Contents: 0..."


In [21]:
df['seller_name'].fillna('Null',inplace= True)
df['seller_name'].isnull().sum()

0

In [22]:
# Inspecting the selller_rating column
df[df['seller_rating'].isnull()]

,category_1,category_2,category_3,title,product_rating,selling_price(KSH.),mrp(KSH.),seller_name,seller_rating,description,highlights
40,"Sports, Books and More",Sports,Badminton,VICTOO SPORTS VICTOO PROFESIONAL BADMINTON Bad...,3.8,254.60,456.00,Null,NaN,NaN,"Sport Type: Badminton Ideal For: Boys, Girls, ..."
310,"Sports, Books and More",Exercise Fitness,Cardio Equipment,Other,0,0.00,0.00,Null,NaN,NaN,NaN
600,"Sports, Books and More",Food Essentials,Nuts and Dry Fruits,OPEN SECRET Snacks Combo | Nutty Cookies and N...,2.7,345.80,0.00,Null,NaN,Nutty Spread - Choco Almond TASTY & HEALTHY: T...,NaN
607,"Sports, Books and More",Food Essentials,Nuts and Dry Fruits,HERBALIFE Weight Loss Combo pack Formula 1 + P...,0,2059.60,3401.76,Null,NaN,Formula 1 Nutri nutritional requirements. Pack...,vitamins minerals protein
681,"Sports, Books and More",Food Essentials,Chocolates,Other,0,0.00,0.00,Null,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11792,Home and Furniture,Home Lighting,Table Lamp,"META White Table Lamp Table Lamp (56 cm, White)",0,2279.24,3800.00,Null,NaN,"White Table lamp for home,bedroom,living room,...","Color: White, Gold Power Source: AC Adapter W ..."
11817,Home and Furniture,Home Lighting,Ceiling Lamp,Other,0,0.00,0.00,Null,NaN,NaN,NaN
11882,Home and Furniture,Festive Decor and Gifts,Festive Decor and Gifts,SHANKAR PERFUMEERY WORKS Combo Pack - Aromatic...,4.1,209.00,0.00,Null,NaN,NaN,Type: Incense Sticks Fragrance: A fresh fragra...
11904,Home and Furniture,Festive Decor and Gifts,Festive Decor and Gifts,UNIBIC Season's Greetings|Christmas & New Year...,3.6,455.24,759.24,Null,NaN,"Season's Greetings, Christmas & New Year Gift,...","Includes: Cake, Chocolate, Cookies Contents: 0..."


In [23]:
df['seller_rating'].fillna('0',inplace=True)
df['seller_rating'].isnull().sum()

0

In [24]:
# Dropping columns not needed for the analysis
df.drop(['description'],axis=1,inplace=True)
df.drop(['highlights'],axis=1,inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   category_1           12041 non-null  object 
 1   category_2           12041 non-null  object 
 2   category_3           12041 non-null  object 
 3   title                12041 non-null  object 
 4   product_rating       12041 non-null  object 
 5   selling_price(KSH.)  12041 non-null  float64
 6   mrp(KSH.)            12041 non-null  float64
 7   seller_name          12041 non-null  object 
 8   seller_rating        12041 non-null  object 
dtypes: float64(2), object(7)
memory usage: 846.8+ KB


In [26]:
df.shape

(12041, 9)

In [27]:
df.category_3.value_counts()[:10]

category_3
Innerwear              160
T -Shirts              120
Shorts                 120
Sport Shoes             80
Jeans                   80
Ethnic Wear             80
Sports Shoes            80
Casual Shoes            80
Sunglasses              80
Luggage and Travel      80
Name: count, dtype: int64

In [28]:
df.category_1.value_counts()

category_1
Women's wear              2422
Men's wear                2360
Bady and Kids             2338
Home and Furniture        2120
Sports, Books and More    1820
Electronics                981
Name: count, dtype: int64

In [29]:
df.head(10)

,category_1,category_2,category_3,title,product_rating,selling_price(KSH.),mrp(KSH.),seller_name,seller_rating
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,1227.40,3040.00,I2SERVICES,4.4
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,115.52,456.00,I2SERVICES,4.4
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,44.84,151.24,VictoryOutlets,4.7
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,57.00,227.24,VictoryOutlets,4.7
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,250.04,1063.24,IndiaFit,4.7
5,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,57.00,227.24,VictoryOutlets,4.7
6,"Sports, Books and More",Sports,Cricket,Ceat six hitman Bat with COVER Poplar Willow C...,3.5,250.04,1744.20,IndiaFit,4.7
7,"Sports, Books and More",Sports,Cricket,VECTOR X Cricket Bat Grip Handle Chevron Extra...,4.1,280.44,300.96,SOCCERINTERNATIONAL,4.1
8,"Sports, Books and More",Sports,Cricket,LYCAN Full Size Pvc/ Hard Plastic Cricket Bat ...,4.0,189.24,835.24,sellguru,4.8
9,"Sports, Books and More",Sports,Cricket,LYCAN Junior Cricket Bat Size 3 For Age Group ...,3.9,189.24,0.00,sellguru,4.8


In [30]:
df['category_1'].value_counts()

category_1
Women's wear              2422
Men's wear                2360
Bady and Kids             2338
Home and Furniture        2120
Sports, Books and More    1820
Electronics                981
Name: count, dtype: int64

In [31]:
df.columns

Index(['category_1', 'category_2', 'category_3', 'title', 'product_rating',
       'selling_price(KSH.)', 'mrp(KSH.)', 'seller_name', 'seller_rating'],
      dtype='object')